> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 4.2 - APIs


## Exercícios 🔭🌌🪐

Vamos acessar as APIs da NASA para ver algumas imagens interessantes capturadas universo afora!

#### Q1.
Crie uma chave no site oficial:
* https://api.nasa.gov

Vamos armazenar a chave de forma segura! <br>
Salve a sua chave em um arquivo `key.json` na forma:
`API_KEY=SUA_CHAVE`

Adicione o nome do arquivo `key.json` ao `.gitignore` do repositório que você fará upload da atividade.
Para isso basta abrir o arquivo `.gitignore` na pasta raíz do repositório (ou criar um caso ele não exista). Dentro do arquivo, apenas adiciona o nome do arquivo que deseja ignorar.

#### Q2. 🛰 Astronomy Picture of the Day (APOD) 🌌
> Antes de fazer os exercícios, devo te lembrar que existem limites de acesso às APIs, descritas na página principal, portanto pega leve na tentativa e erro na hora de testar seu código.

<img width=500 src=https://apod.nasa.gov/apod/image/2407/M24-HaLRGB-RC51_1024.jpg>

A primeira API que acessaremos é a mais popular de todas: astronomy picture of the day (foto astronômica do dia).

Faça uma requisição GET para a URL da API que retorna a imagem do dia! Essa é fácil já que são os valores padrão da rota principal:
* URL base: `'https://api.nasa.gov/planetary/apod'`
* Endpoint: não precisa preencher, acessaremos a raíz da API.
* Query params: preencha `api_key` com a sua chave de autenticação. Se animar mexer em outros parâmetros veja [a documentação](https://api.nasa.gov).

Ao receber a resposta (um json), você deve:
* Imprimir os campos `copyright` e `explanation`
* Com as biblioteca scikit-images e matplotlib, apresente a imagem a partir do campo `url` ou `hdurl`, e preencha o título do plot com o campo `title` do json. Uma dica de código a seguir.
```python
from skimage import io
img = io.imread(url)
## plot a matriz img com matplotlib (imshow)
```   

In [ ]:
import requests
from skimage import io
import matplotlib.pyplot as plt

# Configuração da requisição
url = "https://api.nasa.gov/planetary/apod"
params = {
    "api_key": "t2Jmz9iWaY4DNvbPt4QMxUZv17vLydvMVPd1nx9k"  # Minha chave 
}

# Fazendo a requisição GET
response = requests.get(url, params=params)

# Verificando a resposta
if response.status_code == 200:
    data = response.json()
    
    # Imprimindo os campos solicitados
    print("Copyright:", data.get("copyright", "Desconhecido"))
    print("\nExplanation:", data.get("explanation", "Sem explicação disponível"))
    
    # Obtendo a URL da imagem (usa hdurl se disponível, caso contrário url)
    image_url = data.get("hdurl", data.get("url"))
    
    if image_url and data.get("media_type") == "image":
        # Carregando e exibindo a imagem
        img = io.imread(image_url)
        
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.title(data.get("title", "Astronomy Picture of the Day"))
        plt.axis('off')  # Remove os eixos
        plt.show()
    else:
        print("\nHoje não é uma imagem, mas um vídeo. URL do vídeo:", data.get("url"))
else:
    print("Erro na requisição:", response.status_code)
    print(response.text)

#### Q3. Limites
A partir da resposta da query anterios, imprima o header da resposta e consulte os atributos:
* X-RateLimit-Limit: o limite total de requisições da sua chave de API
* X-RateLimit-Remaining: o limite restante de requisições da sua chave de API

In [ ]:
import requests

# Minha chave de API 
API_KEY = "t2Jmz9iWaY4DNvbPt4QMxUZv17vLydvMVPd1nx9k"

# URL da API APOD
url = "https://api.nasa.gov/planetary/apod"

# Parâmetros da requisição
params = {
    "api_key": API_KEY
}

# Fazendo a requisição GET
response = requests.get(url, params=params)

# Imprimindo todos os headers da resposta
print("Headers da resposta:")
print(response.headers)
print("\n")

# Consultando os atributos específicos de rate limiting
rate_limit = response.headers.get("X-RateLimit-Limit")
remaining = response.headers.get("X-RateLimit-Remaining")

print(f"Limite total de requisições (X-RateLimit-Limit): {rate_limit}")
print(f"Requisições restantes (X-RateLimit-Remaining): {remaining}")

# Verificando se a requisição foi bem sucedida
if response.status_code == 200:
    data = response.json()
    print("\nDados da APOD:")
    print(f"Título: {data.get('title')}")
    print(f"Copyright: {data.get('copyright', 'Desconhecido')}")
else:
    print("\nErro na requisição:", response.status_code)
    print(response.text)

### Q4. Mars Rover Photos 🚀🚙 📷

<img width=500 src=https://www.nasa.gov/wp-content/uploads/2019/10/pia23378-16.jpg>

Essa API retorna dados (incluindo imagens capturadas) sobre os veículos que hoje habitam o planeta Marte. São os rovers `opportunity`, `spirit` e o mais famoso, o `curiosity` (da foto acima).

Antes de requisitar imagens, vamos ver o relatório de dados coletados por um deles, o `curiosity`. Isso vai nos ajudar a montar a query de imagens coletadas.

Faça uma requisição GET para a seguinte URL:
* URL base: `'https://api.nasa.gov/mars-photos/api/v1'`
* endpoint: `'/manifests/{nome_do_rover}'`
* query parameters: preencha `api_key` com a sua chave de autenticação.

Extraia o json da resposta retornada. O campo principal é o `'photo_manifest'`, do qual queremos acessar os seguintes valores:
* `max_sol`: Máximo "dia marciano" de coleta de fotos. O dia marciano tem 24 horas, 39 minutos e 35 segundos.
* `max_date`: Última data terrestre de coleta de fotos, na forma `'aaaa-mm-dd'`.

Imprima esses dois atributos da resposta e os use no próximo exercício para coletar as fotos mais recentes tiradas. 

In [ ]:
import requests

# Configuração da requisição
BASE_URL = 'https://api.nasa.gov/mars-photos/api/v1'
ENDPOINT = '/manifests/curiosity'
API_KEY = 't2Jmz9iWaY4DNvbPt4QMxUZv17vLydvMVPd1nx9k'  

# Montando a URL completa
url = f"{BASE_URL}{ENDPOINT}"

# Parâmetros da requisição
params = {
    'api_key': API_KEY
}

# Fazendo a requisição GET
response = requests.get(url, params=params)

# Verificando a resposta
if response.status_code == 200:
    data = response.json()
    manifest = data.get('photo_manifest', {})
    
    # Extraindo os valores solicitados
    max_sol = manifest.get('max_sol')
    max_date = manifest.get('max_date')
    
    # Imprimindo os resultados
    print(f"Último dia marciano (sol) de coleta: {max_sol}")
    print(f"Última data terrestre de coleta: {max_date}")
    
    # Retornando os valores para uso posterior
    curiosity_data = {
        'max_sol': max_sol,
        'max_date': max_date
    }
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)
    curiosity_data = None

#### Q5.

Faça uma requisição GET para a URL da API que retorna links para as imagens coletadas pelos rovers.

* URL base: `'https://api.nasa.gov/mars-photos/api/v1'`
* Endpoint: `/rovers/{nome_do_rover}/photos`
* Query params sugeridos: 
    * `api_key`: sua chave de autenticação.
    * `sol`: dia marciano que deseja coletar (de 0 a `max_sol` coletado anteriormente)
    * `page`: você pode paginar entre as respostas! São retornados 25 resultados por página.

A resposta esperada estará no formato a seguir, uma lista no campo `'photos'` onde cada item é um dicionário com os dados da foto tirada. Dentre os dados há o campo `camera` indicando qual das câmeras do rover tirou a foto. As fotos mais interessantes (na minha opinião, claro) são das câmeras de navegação (`"name": "NAVCAM"`) e as de prevenção de colisão (frente: `"name": "FHAZ"` e trás `"name": "RHAZ"`) onde dá pra ver partes do robô!

**Seu trabalho é**:
* Paginar a requisição acima até que a resposta seja `None`
* Escolher uma ou mais câmeras (ex: `NAVCAM`, `FHAZ`, `RHAZ`), e em um laço de repetição plotar todas as imagens retornadas daquela câmera. Use novamente as bibliotecas scikit-image e matplotlib. 
  * O título da imagem deve ter a página da requisição, nome da câmera e id da imagem.

```json
{
  "photos": [
    {
      "id": 1228212,
      "sol": 4102,
      "camera": {
        "id": 20,
        "name": "FHAZ",
        "rover_id": 5,
        "full_name": "Front Hazard Avoidance Camera"
      },
      "img_src": "https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/04102/opgs/edr/fcam/FLB_761645828EDR_F1060660FHAZ00302M_.JPG",
      "earth_date": "2024-02-19",
      "rover": {
        "id": 5,
        "name": "Curiosity",
        ...
      }
    }
    {
      "id": 1228213,
      "sol": 4102, 
      ...
    }
```



In [ ]:
import requests
from skimage import io
import matplotlib.pyplot as plt

# Configurações iniciais
BASE_URL = 'https://api.nasa.gov/mars-photos/api/v1'
ROVER = 'curiosity'
API_KEY = 't2Jmz9iWaY4DNvbPt4QMxUZv17vLydvMVPd1nx9k' # Minha chave
CAMERAS = ['NAVCAM', 'FHAZ', 'RHAZ']  # Câmeras que queremos visualizar
SOL = 4102  # Dia marciano (pode usar o max_sol obtido anteriormente)

def get_rover_photos(page=1):
    """Faz requisição para a API de fotos do rover"""
    endpoint = f'/rovers/{ROVER}/photos'
    url = f"{BASE_URL}{endpoint}"
    
    params = {
        'api_key': API_KEY,
        'sol': SOL,
        'page': page
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    return None

def display_photos():
    """Busca e exibe todas as fotos das câmeras selecionadas"""
    page = 1
    while True:
        data = get_rover_photos(page)
        if not data or 'photos' not in data or not data['photos']:
            break
            
        print(f"\nProcessando página {page}...")
        
        for photo in data['photos']:
            camera_name = photo['camera']['name']
            
            if camera_name in CAMERAS:
                try:
                    img = io.imread(photo['img_src'])
                    plt.figure(figsize=(10, 8))
                    plt.imshow(img)
                    plt.title(f"Pág {page} | {camera_name} | ID: {photo['id']}\n"
                             f"Sol: {photo['sol']} | Data: {photo['earth_date']}")
                    plt.axis('off')
                    plt.show()
                except Exception as e:
                    print(f"Erro ao carregar imagem ID {photo['id']}: {e}")
        
        page += 1

# Executando a função principal
display_photos()